In [26]:
%load_ext Cython
%time
%cython

ERROR:root:Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


In [27]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [28]:
import sqlite3
conn = sqlite3.connect('data/capstone.db')
# tblGame , tblMovie, tblReview, tblTVShow

In [29]:
# load games dataset
games = pd.read_sql_query("select * from tblGame;", conn)
games['gameID'] = games.index
games.head(5)

,name,link,image,developer,genre,rating,rlsDate,summary,gameID
0,Ginger: Beyond the Crystal,/game/playstation-4/ginger-beyond-the-crystal,http://static.metacritic.com/images/products/g...,Drakhar Studio,Action,E,2016-10-25 00:00:00,"A world destroyed by a crystal explosion, a my...",0
1,JoJo's Bizarre Adventure: Eyes of Heaven,/game/playstation-4/jojos-bizarre-adventure-ey...,http://static.metacritic.com/images/products/g...,CyberConnect2,Action,T,2016-06-28 00:00:00,"Featuring a new two-on-two battle system, larg...",1
2,The Crew,/game/playstation-4/the-crew,http://static.metacritic.com/images/products/g...,"Ubisoft Reflections, Ivory Tower",Racing,T,2014-12-02 00:00:00,Strap in for a ride that will find you infiltr...,2
3,Batman: Arkham Knight - A Matter of Family,/game/playstation-4/batman-arkham-knight---a-m...,http://static.metacritic.com/images/products/g...,"Rocksteady Studios, WB Games Montreal",Action Adventure,M,2015-07-14 00:00:00,When The Joker kidnaps Commissioner Gordon and...,3
4,8DAYS,/game/playstation-4/8days,http://static.metacritic.com/images/products/g...,Santa Clara Games,Action,M,2017-02-07 00:00:00,The world has been dominated by dark forces si...,4


In [36]:
# Read in Critic Reviews
reviews = pd.read_sql_query("select * from tblReview;", conn)


In [34]:
# subset for just critic reviews to build a model
subset = reviews[reviews['reviewType']=='c']
subset = df[['gameID','text']]
subset = subset.dropna()
subset.head(5)

In [38]:
# get a unique review corpus
review_docs = subset.groupby(['gameID'])['text'].apply(lambda x: ''.join(x)).reset_index()
review_docs.head(5)

,gameID,text
0,1.0,Ginger: Beyond the Crystal is a very good plat...
1,2.0,JoJo’s Bizarre Adventure: Eyes in Heaven featu...
2,3.0,The Crew does a lot of things right. Not only ...
3,4.0,Playing as Batgirl is not quite as interesting...
4,5.0,The frantic nature of the gameplay coupled wit...


In [39]:
stopwrds = stopwords.words('english')

In [40]:
# aux function to clean up text
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]',' ', sentence)
    sentence = re.sub('_',' ', sentence)
    sentence = re.sub('\d+',' ', sentence)
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])
    cleaned = cleaned.strip()
    return cleaned

In [41]:
# clean up review docs, add utf8 encoding
review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)

In [42]:
# define a few functions for doc2vec processing
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(review_docs.shape[0]):
            yield doc2vec.LabeledSentence(words=split_sentence(review_docs.iloc[i,2]), tags=['%s' % i])

In [43]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save("data/review.model")

In [44]:
# testing similar words
print model.most_similar(positive=["cheap", "zelda"], negative=["slow"], topn=5)

[('princess', 0.5611832141876221), ('inexpensive', 0.5419542789459229), ('deluxe', 0.5292503833770752), ('eshop', 0.5130327343940735), ('justifiable', 0.5031678676605225)]


In [45]:
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a dataframe with top_n products with cosine similarity score
def recommend(l,n):
    # get indexes of top_n similar items 
    index = top_n_index(l, n)
    
    # return list of products 
    top_products = copy.deepcopy(review_docs.iloc[index,][['gameID']])
    top_products['index']= top_products.index
    
    # initialize an empty column for cosine similarity
    top_products['cossim']=0
    
    for i in range(len(index)):
        # note - this results in a bug: top_products['cossim'].loc[index[i],] = sim_array[index[i]]
        top_products.loc[index[i],'cossim'] = sim_array[index[i]]
        
    return top_products   

In [46]:
games.loc[games['name'].str.contains('Mario')]

,name,link,image,developer,genre,rating,rlsDate,summary,gameID
396,Mario Tennis: Ultra Smash,/game/wii-u/mario-tennis-ultra-smash,http://static.metacritic.com/images/products/g...,"Nintendo, Camelot Software Planning",Sports,E,2015-11-20 00:00:00,Compete with cohorts in the multiplayer Mario ...,396
681,Mario & Sonic at the Sochi 2014 Olympic Winter...,/game/wii-u/mario-sonic-at-the-sochi-2014-olym...,http://static.metacritic.com/images/products/g...,"Nintendo, Sega Sports R&D",Sports,E,2013-11-15 00:00:00,Join Mario and Sonic and the rest of your favo...,681
835,Super Mario Sunshine,/game/gamecube/super-mario-sunshine,http://static.metacritic.com/images/products/g...,Nintendo,Action,E,2002-08-25 00:00:00,"When Mario, Peach, and an entourage of Peach\s...",835
850,Mario Kart DS,/game/ds/mario-kart-ds,http://static.metacritic.com/images/products/g...,Nintendo,Driving,E,2005-11-14 00:00:00,Race and battle with up to eight karts at once...,850
870,Mario & Luigi: Bowser's Inside Story,/game/ds/mario-luigi-bowsers-inside-story,http://static.metacritic.com/images/products/g...,Alphadream Corporation,Role-Playing,E,2009-09-14 00:00:00,In his evil plot to take over the Mushroom Kin...,870
950,New Super Mario Bros.,/game/ds/new-super-mario-bros,http://static.metacritic.com/images/products/g...,Nintendo,Action,E,2006-05-15 00:00:00,The first new 2D Mario platformer since Super ...,950
1000,Mario Party 6,/game/gamecube/mario-party-6,http://static.metacritic.com/images/products/g...,Hudson Soft,Miscellaneous,E,2004-12-06 00:00:00,Mario Party 6 offers a new batch of games that...,1000
1262,Mario & Luigi: Partners in Time,/game/ds/mario-luigi-partners-in-time,http://static.metacritic.com/images/products/g...,Alphadream Corporation,Role-Playing,E,2005-11-28 00:00:00,In this sequel to Mario & Luigi: Superstar Sag...,1262
1389,Super Mario Galaxy,/game/wii/super-mario-galaxy,http://static.metacritic.com/images/products/g...,Nintendo,Action,E,2007-11-12 00:00:00,[Metacritic\s 2007 Wii Game of the Year] The u...,1389
1417,Super Mario Galaxy 2,/game/wii/super-mario-galaxy-2,http://static.metacritic.com/images/products/g...,Nintendo EAD Tokyo,Action,E,2010-05-23 00:00:00,"Super Mario Galaxy 2, the sequel to the galaxy...",1417


In [49]:
# "Basic recommendation system with doc2vec

# Select gameID of a product you like
gameID = 950
input_vec = model.docvecs[review_docs.loc[review_docs['gameID']==gameID].index[0]]

# Calculated a cossim matrix between input product and model's vectors
sim_array = map(lambda v: cossim(input_vec, v), model.docvecs)

#recommendation
print "You liked this product::: %s" %games.loc[games['gameID']==gameID]['name']
res = recommend(sim_array,5)
res['gameID'] = res.apply(lambda row: int(row['gameID']),axis = 1)
pd.merge(res, games[['gameID','name']], on='gameID')

You liked this product::: 950    New Super Mario Bros.
Name: name, dtype: object


,gameID,index,cossim,name
0,15329,15302,0.744762,Dementium II
1,739,738,0.755695,SimCity Creator
2,16530,16498,0.756579,Dead Effect 2
3,644,643,0.777793,Disney's Treasure Planet
4,6297,6294,0.778833,Real World Golf


In [56]:
#save a pickle file
import pickle

with open('models/model_critic_only.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)
